# Analisando a evolução das quadras fiscais

A referência espacial para este estudo serão as quadras fiscais, pois sabemos que os lotes fiscais são mais sujeitos a remembramentos e desdobros, enquanto as quadras fiscais seguem um fluxo de atualização menos frequente.

Portanto o objetivo deste script é a partir dos conjuntos de dados de IPTU desde 1995 quantificar e identificar os setores que sofreram alteração para validar o método de espacialização.

## Carregando dados de cada um dos anos de IPTU

Para cada conjunto de dados de IPTU:

* Carregar os dados
* Obter SQ (Setor/Quadra)
* Agregar (apenas contagem) por SQ
* Anotar o Ano de Exercício
* Adicionar À lista

E por último salvar o resultado da lista com todos os anos

In [2]:
# Carrengando um complemento para visualizar quanto tempo demoram as tarefas

%load_ext autotime

In [3]:
from pathlib import Path
import os
import pandas as pd

In [10]:
sqs_iptus = []

for path in Path('IPTU').rglob('*.zip'):
    print(path.name)
    
    # 1. Carregar dados do arquivo
    # OBS.: O IPTU de 2016 está separado por ',' e não por ';'
    if path.name == "IPTU_2016.zip":
        df_iptu = pd.read_csv(f'{path}',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=',',\
                     decimal=',',\
                     error_bad_lines=False)
    else:
        df_iptu = pd.read_csv(f'{path}',\
                     compression='zip',\
                     encoding='iso-8859-9',\
                     sep=';',\
                     decimal=',',\
                     error_bad_lines=False)
            
    # 2. Obter o SQ (Setor e Quadra)
    df_iptu['setor'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[0:3]
    df_iptu['quadra'] = df_iptu['NUMERO DO CONTRIBUINTE'].str[3:6]
    
    # 3. Agregar por SQ e ANO DO EXERCICIO
    iptu_agg = df_iptu[['setor', 'quadra', 'NUMERO DO CONTRIBUINTE', 'ANO DO EXERCICIO']]\
    .groupby(['setor', 'quadra', 'ANO DO EXERCICIO'])\
    .agg({'NUMERO DO CONTRIBUINTE': 'count'})\
    .rename(columns = {'NUMERO DO CONTRIBUINTE': 'quantidade'})

    # adicionando à lista
    sqs_iptus.append(iptu_agg)


IPTU_2010.zip


b'Skipping line 2445507: expected 35 fields, saw 43\n'
/home/fernando/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


IPTU_1996.zip


/home/fernando/miniconda3/envs/geo/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


IPTU_2007.zip


...

b'Skipping line 2440049: expected 35 fields, saw 43\n'


IPTU_2003.zip


b'Skipping line 2288886: expected 35 fields, saw 43\n'


IPTU_2012.zip


b'Skipping line 2526350: expected 35 fields, saw 43\n'


In [15]:
pd.concat(sqs_iptus).to_csv('resultados/sqs_iptu_por_ano.csv')

In [16]:
df_sqs_iptus = pd.concat(sqs_iptus)

In [18]:
df_sqs_iptus.groupby('ANO DO EXERCICIO').agg('count')

,quantidade
ANO DO EXERCICIO,
1995,41950
1996,42019
1997,42168
1998,42368
1999,42392
2000,42396
2001,42688
2002,42669
2003,42709


In [78]:
import geopandas as gpd

In [79]:
gdf_quadras = gpd.GeoDataFrame.from_file(f'zip://GIS/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF')

In [80]:
gdf_quadras.qd_tipo.value_counts()

F    54790
M     5841
R     2948
E       36
Name: qd_tipo, dtype: int64

In [81]:
gdf_quadras = gdf_quadras.loc[(gdf_quadras.qd_tipo == 'F') | (gdf_quadras.qd_tipo == 'R') ]

In [82]:
gdf_quadras['sq'] = gdf_quadras['qd_setor'] + gdf_quadras['qd_fiscal']

/home/fernando/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [83]:
gdf_quadras_dissolved = gdf_quadras.dissolve(by='sq', aggfunc='first')

In [84]:
gdf_quadras_dissolved.reset_index(inplace=True)

In [85]:
df_sqs_iptus.reset_index(inplace=True)

In [86]:
df_sqs_iptus['sq'] = df_sqs_iptus.setor + df_sqs_iptus.quadra

In [87]:
df_sqs_iptus.merge(gdf_quadras_dissolved, on='sq', how='left').geometry.isna().value_counts()

False    1120383
True        5128
Name: geometry, dtype: int64

In [88]:
df_sqs_iptus[df_sqs_iptus.merge(gdf_quadras_dissolved, on='sq', how='left').geometry.isna()]\
['ANO DO EXERCICIO'].value_counts()

1995    341
1996    338
1997    312
1998    305
1999    295
2001    283
2000    282
2003    254
2002    252
2004    245
2005    229
2006    205
2007    190
2008    166
2009    149
2010    143
2011    142
2012    134
2013    129
2014    114
2019    111
2016    109
2017    109
2020    109
2015    107
2018     75
Name: ANO DO EXERCICIO, dtype: int64